In [1]:
import os
import subprocess
from ipywidgets import widgets, VBox, Button, Output
from tqdm.notebook import tqdm
from IPython.display import display

# Define paths to your atlas files and MNI space file
atlas_paths = {
    'network': '/media/mm/Mahdi-2022/Atlas/Networks_Atlas/networks_ROIs.nii',
    'visual': '/media/mm/Mahdi-2022/Atlas/Visual_Atlas/visual_ROIs.nii',
    'motor': '/media/mm/Mahdi-2022/Atlas/Motor_Atlas/motor_ROIs.nii' 

}
mni_path = '/media/mm/Mahdi-2022/Atlas/bet_mni.nii' 

def mirtk_registration(patient_ids, register_network, register_visual, register_motor):
    for patient_id in patient_ids.split(','):  # Splitting the input string by commas
        patient_dir = f'/path/to/directory/{patient_id.strip()}/dti_outputs'  # .strip() to remove any leading/trailing whitespace
        os.chdir(patient_dir)

        b0_nii_path = 'b0.nii'
        b0_exists = os.path.exists(b0_nii_path)

        total_steps = 3 + sum([register_network, register_visual, register_motor])  # Initial steps + conditional atlas registrations
        progress = tqdm(total=total_steps, desc=f'Total Progress for {patient_id.strip()}')

        # Check if b0.nii already exists; if not, convert b0.mif to b0.nii
        if not b0_exists:
            subprocess.run(['mrconvert', 'b0.mif', b0_nii_path])
            progress.update(1)  # Update progress after conversion
        else:
            progress.update(1)  # Update progress, acknowledging that b0.nii was already present

        # BET extraction
        subprocess.run(['bet', b0_nii_path, 'bet_b0.nii', '-m', '-f', 0.25])
        progress.update(1)  # Update progress after BET extraction

        # MNI registration
        subprocess.run(['mirtk', 'register', '-image', 'bet_b0.nii.gz', mni_path,
                        '-dofout', 'transfomation.txt', '-output', 'r_mni.nii'])
        progress.update(1)  # Update progress after MNI registration

        # Perform atlas registrations based on user input
        for atlas_type, do_register in [('network', register_network), 
                                        ('visual', register_visual), 
                                        ('motor', register_motor)]:
            if do_register:
                atlas_path = atlas_paths[atlas_type]
                output_path = f'r_{atlas_type}_ROI.nii'
                subprocess.run(['mirtk', 'transform-image', atlas_path, output_path,
                                '-target', 'bet_b0.nii.gz', '-dofin', 'transfomation.txt', '-interp', 'NN'])
                progress.update(1)  # Update progress after each atlas registration

        progress.close()  # Close the progress bar when done
        print(f'Registration complete for patient: {patient_id.strip()}')

# Interactive Widgets
patient_id_widget = widgets.Text(value='patient_id1, patient_id2', description='Patient IDs:', disabled=False)
network_widget = widgets.Checkbox(value=False, description='Register Network Areas', disabled=False)
visual_widget = widgets.Checkbox(value=False, description='Register Visual Areas', disabled=False)
motor_widget = widgets.Checkbox(value=False, description='Register Motor Areas', disabled=False)

ui = VBox([patient_id_widget, network_widget, visual_widget, motor_widget])
out = Output()

def on_button_clicked(_):
    with out:
        out.clear_output()
        mirtk_registration(patient_id_widget.value, network_widget.value, visual_widget.value, motor_widget.value)

button = Button(description="Run Registration")
button.on_click(on_button_clicked)

display(ui, button, out)

Button(description='Run Registration', style=ButtonStyle())

Output()

In [4]:
import os
import subprocess
from ipywidgets import widgets, VBox, Button, Output
from tqdm.notebook import tqdm
from IPython.display import display

# Define paths to your atlas files and MNI space file
atlas_paths = {
    'network': '/media/mm/Mahdi-2022/Atlas/Networks_Atlas/networks_ROIs.nii',
    'visual': '/media/mm/Mahdi-2022/Atlas/Visual_Atlas/visual_ROIs.nii',
    'motor': '/media/mm/Mahdi-2022/Atlas/Motor_Atlas/motor_ROIs.nii' 

}
mni_path = '/media/mm/Mahdi-2022/Atlas/bet_mni.nii' 

def mirtk_registration(patient_ids, register_network, register_visual, register_motor):
    base_path = '/media/mm/Mahdi-2022/Laxman_PD_Project/MAP_complete'  # Base path for patient data
    for patient_id in patient_ids.split(','):  # Splitting the input string by commas
        patient_dir = f'{base_path}/{patient_id.strip()}/dti_outputs'  # Construct the full path to dti_outputs
        if not os.path.exists(patient_dir):
            print(f"Directory does not exist: {patient_dir}")
            continue  # Skip this patient if the directory does not exist
        os.chdir(patient_dir)
    


        b0_nii_path = 'b0.nii'
        b0_exists = os.path.exists(b0_nii_path)

        total_steps = 3 + sum([register_network, register_visual, register_motor])  # Initial steps + conditional atlas registrations
        progress = tqdm(total=total_steps, desc=f'Total Progress for {patient_id.strip()}')

        # Check if b0.nii already exists; if not, convert b0.mif to b0.nii
        if not b0_exists:
            subprocess.run(['mrconvert', 'b0.mif', b0_nii_path])
            progress.update(1)  # Update progress after conversion
        else:
            progress.update(1)  # Update progress, acknowledging that b0.nii was already present

        # BET extraction
        subprocess.run(['bet', b0_nii_path, 'bet_b0.nii', '-m', '-f', '0.25'])
        progress.update(1)  # Update progress after BET extraction

        # MNI registration
        subprocess.run(['mirtk', 'register', '-image', 'bet_b0.nii.gz', mni_path,
                        '-dofout', 'transfomation.txt', '-output', 'r_mni.nii'])
        progress.update(1)  # Update progress after MNI registration



        # Perform atlas registrations based on user input
        for atlas_type, do_register in [('network', register_network), 
                                        ('visual', register_visual), 
                                        ('motor', register_motor)]:
            if do_register:
                atlas_path = atlas_paths[atlas_type]
                output_path = f'r_{atlas_type}_ROI.nii'
                subprocess.run(['mirtk', 'transform-image', atlas_path, output_path,
                                '-target', 'bet_b0.nii.gz', '-dofin', 'transfomation.txt', '-interp', 'NN'])
                progress.update(1)  # Update progress after each atlas registration

        progress.close()  # Close the progress bar when done
        print(f'Registration complete for patient: {patient_id.strip()}')


patient_folders = [
    "Acchione_Colleen", "Grandinetti_Christine", "PerezCepedes_Jose",
    "Adams_JoshuaAnthony", "Guistwite_Edward", "Petekiewicz_Anne",
    "Aezen_Peter", "Hacker_Susan", "Piraine_Phyllis",
    "Allen_Eric", "Hamilton_Gary", "Poley_Daniel",
    "Allen_Eric_2", "Hamilton_Gary_2", "Reynolds_Christopher",
    "Baches_Matthew", "Harold_James", "Rhea_Byron",
    "Bach_Nancy", "Harrison_Christopher", "Rhynalds_Donna",
    "Benedetti_Eric", "Jerome_Patricia", "Richard_Craig",
    "Bergh_Charles", "Lapinson_Craig", "Rivera_Eladio",
    "Beuhler_WadeAllen", "Lindsey_Daniel", "Robin_Foy",
    "Blackmon_Whitney", "Lyons_Connie", "Ruben_Nicholas",
    "Bono_Michael", "Mallya_Satish", "Sabutis_Charles",
    "Boon_Maurits", "March_MaryDianne", "Samuel_Nicki",
    "Cerreta_MaryAlice", "McIntyre_William", "Schilling_Dale",
    "Charrington_Arthur", "Meyers_William", "Schwartz_Louis",
    "Chatterjee_Abhijit", "Miller_JamesFargo", "Scully_Thomas",
    "Cook_Frank", "Miranda_Wayne", "Shah_Meera",
    "Curran_Denise", "Morina_Alketa", "Simonete_Andrade",
    "Dowe_Alfonzo", "Nagy_Richard", "Smith_David",
    "Dunigan_Loretta", "Neavil_Susan", "Smith_Robert",
    "Dutton_Rosalind", "Neff_Marc", "Venette_Frederic",
    "Endres_Gina", "Nussbaum_Lawrence", "Wayne_King",
    "Ennis_Carlton", "Pack_Sun", "Winslow_William",
    "Fritsche_TeresaKeyek", "Parker_Karen", "Winsor_Nancy",
    "Fritsche_TeresaKeyek_2", "Patel_Maheshkumar", "Garrison_Lisa",
    "PattersonCrouse_John"
]


# Interactive Widgets
patient_id_widget = widgets.Text(value='patient_id1, patient_id2', description='Patient IDs:', disabled=False)
network_widget = widgets.Checkbox(value=False, description='Register Network Areas', disabled=False)
visual_widget = widgets.Checkbox(value=False, description='Register Visual Areas', disabled=False)
motor_widget = widgets.Checkbox(value=False, description='Register Motor Areas', disabled=False)

ui = VBox([patient_id_widget, network_widget, visual_widget, motor_widget])
out = Output()

def on_button_clicked(_):
    with out:
        out.clear_output()
        mirtk_registration(patient_id_widget.value, network_widget.value, visual_widget.value, motor_widget.value)

button = Button(description="Run Registration")
button.on_click(on_button_clicked)

display(ui, button, out)

Button(description='Run Registration', style=ButtonStyle())

Output()

In [6]:
import os
import subprocess
from tqdm import tqdm  # Ensure to use the non-notebook version of tqdm for scripts

# Define paths to your atlas files and MNI space file
atlas_paths = {
    'network': '/media/mm/Mahdi-2022/Atlas/Networks_Atlas/networks_ROIs.nii',
    'visual': '/media/mm/Mahdi-2022/Atlas/Visual_Atlas/visual_ROIs.nii',
    'motor': '/media/mm/Mahdi-2022/Atlas/Motor_Atlas/motor_ROIs.nii'
}
mni_path = '/media/mm/Mahdi-2022/Atlas/bet_mni.nii'

# Define which registrations to perform
register_network = False
register_visual = True
register_motor = False

def mirtk_registration(patient_folders):
    base_path = '/media/mm/Mahdi-2022/Laxman_PD_Project/MAP_complete'  # Base path for patient data
    total_patients = len(patient_folders)
    patient_progress = tqdm(patient_folders, desc='Overall Patient Progress')

    for patient_id in patient_progress:
        patient_dir = f'{base_path}/{patient_id}/dti_outputs'  # Construct the full path to dti_outputs
        if not os.path.exists(patient_dir):
            print(f"Directory does not exist: {patient_dir}")
            continue  # Skip this patient if the directory does not exist
        os.chdir(patient_dir)
        
        print(f"Starting registration for {patient_id}")
        b0_nii_path = 'b0.nii'
        b0_exists = os.path.exists(b0_nii_path)

        total_steps = 3 + sum([register_network, register_visual, register_motor])  # Initial steps + conditional atlas registrations
        progress = tqdm(total=total_steps, desc=f'Total Progress for {patient_id}', leave=False)

        # Check if b0.nii already exists; if not, convert b0.mif to b0.nii
        if not b0_exists:
            print(f"Converting b0.mif to b0.nii for {patient_id}")
            subprocess.run(['mrconvert', 'b0.mif', b0_nii_path])
        progress.update()  # Update progress after conversion or acknowledge existing file

        # BET extraction
        print(f"Starting BET extraction for {patient_id}")
        subprocess.run(['bet', b0_nii_path, 'bet_b0.nii', '-m', '-f', '0.25'])
        progress.update()  # Update progress after BET extraction

        # MNI registration
        print(f"Starting MNI registration for {patient_id}")
        subprocess.run(['mirtk', 'register', '-image', 'bet_b0.nii.gz', mni_path,
                        '-dofout', 'transfomation.txt', '-output', 'r_mni.nii'])
        progress.update()  # Update progress after MNI registration

        # Perform atlas registrations based on user input
        for atlas_type, do_register in [('network', register_network),
                                        ('visual', register_visual),
                                        ('motor', register_motor)]:
            if do_register:
                print(f"Starting {atlas_type} registration for {patient_id}")
                atlas_path = atlas_paths[atlas_type]
                output_path = f'r_{atlas_type}_ROI.nii'
                subprocess.run(['mirtk', 'transform-image', atlas_path, output_path,
                                '-target', 'bet_b0.nii.gz', '-dofin', 'transfomation.txt', '-interp', 'NN'])
                progress.update()  # Update progress after each atlas registration
                print(f'Registration complete for {patient_id}: {atlas_type}')

        progress.close()  # Close the progress bar when done

# List of patient folders
patient_folders = [
    "Acchione_Colleen", "Grandinetti_Christine", "PerezCepedes_Jose",
    "Adams_JoshuaAnthony", "Guistwite_Edward", "Petekiewicz_Anne",
    "Aezen_Peter", "Hacker_Susan", "Piraine_Phyllis",
    "Allen_Eric", "Hamilton_Gary", "Poley_Daniel",
    "Allen_Eric_2", "Hamilton_Gary_2", "Reynolds_Christopher",
    "Baches_Matthew", "Harold_James", "Rhea_Byron",
    "Bach_Nancy", "Harrison_Christopher", "Rhynalds_Donna",
    "Benedetti_Eric", "Jerome_Patricia", "Richard_Craig",
    "Bergh_Charles", "Lapinson_Craig", "Rivera_Eladio",
    "Beuhler_WadeAllen", "Lindsey_Daniel", "Robin_Foy",
    "Blackmon_Whitney", "Lyons_Connie", "Ruben_Nicholas",
    "Bono_Michael", "Mallya_Satish", "Sabutis_Charles",
    "Boon_Maurits", "March_MaryDianne", "Samuel_Nicki",
    "Cerreta_MaryAlice", "McIntyre_William", "Schilling_Dale",
    "Charrington_Arthur", "Meyers_William", "Schwartz_Louis",
    "Chatterjee_Abhijit", "Miller_JamesFargo", "Scully_Thomas",
    "Cook_Frank", "Miranda_Wayne", "Shah_Meera",
    "Curran_Denise", "Morina_Alketa", "Simonete_Andrade",
    "Dowe_Alfonzo", "Nagy_Richard", "Smith_David",
    "Dunigan_Loretta", "Neavil_Susan", "Smith_Robert",
    "Dutton_Rosalind", "Neff_Marc", "Venette_Frederic",
    "Endres_Gina", "Nussbaum_Lawrence", "Wayne_King",
    "Ennis_Carlton", "Pack_Sun", "Winslow_William",
    "Fritsche_TeresaKeyek", "Parker_Karen", "Winsor_Nancy",
    "Fritsche_TeresaKeyek_2", "Patel_Maheshkumar", "Garrison_Lisa",
    "PattersonCrouse_John"
]


# Call the registration function for all patients
mirtk_registration(patient_folders)


Starting registration for Acchione_Colleen


Starting BET extraction for Acchione_Colleen


Starting MNI registration for Acchione_Colleen
     0.95647     =  0.95396 (     1.23161    ) +  0.00251 (     2.51203    )register  (rev 6461e43, built on Mar 15 2021)


Registration with rigid transformation model

Resolution level 4

          Energy            Image dissimilarity             Bending energy       
   1      1.00000     =  1.00000 (     1.08816    ) +  0.00000 (     0.00000    )
   2      0.94738     =  0.94738 (     1.13614    ) +  0.00000 (     0.00000    )
   3      0.94191     =  0.94191 (     1.14112    ) +  0.00000 (     0.00000    )
   4      0.92675     =  0.92675 (     1.15494    ) +  0.00000 (     0.00000    )
   5      0.91350     =  0.91350 (     1.16703    ) +  0.00000 (     0.00000    )
   6      0.89139     =  0.89139 (     1.18719    ) +  0.00000 (     0.00000    )
   7      0.87475     =  0.87475 (     1.20236    ) +  0.00000 (     0.00000    )
   8      0.86966     =  0.86966 (     1.20701    ) +  0.00000 (     0.00000    )
   9      0.86665     =  

Starting visual registration for Acchione_Colleen


Registration complete for Acchione_Colleen: visual
Starting registration for Grandinetti_Christine


Starting BET extraction for Grandinetti_Christine


Starting MNI registration for Grandinetti_Christine
register  (rev 6461e43, built on Mar 15 2021)


Registration with rigid transformation model

Resolution level 4

          Energy            Image dissimilarity             Bending energy       
   1      1.00000     =  1.00000 (     1.10384    ) +  0.00000 (     0.00000    )
   2      0.95886     =  0.95886 (     1.14070    ) +  0.00000 (     0.00000    )
   3      0.95310     =  0.95310 (     1.14587    ) +  0.00000 (     0.00000    )
   4      0.93423     =  0.93423 (     1.16278    ) +  0.00000 (     0.00000    )
   5      0.92660     =  0.92660 (     1.16961    ) +  0.00000 (     0.00000    )
   6      0.91342     =  0.91342 (     1.18143    ) +  0.00000 (     0.00000    )
   7      0.91107     =  0.91107 (     1.18353    ) +  0.00000 (     0.00000    )
   8      0.91009     =  0.91009 (     1.18441    ) +  0.00000 (     0.00000    )
   9      0.90967     =  0.90967 (     1.18479    ) +  0.00000 (     0.00000    )

Resolution le